In [2]:
# !pip install gensim --user
# get_ipython().system('pip install gensim')

In [3]:
from gensim.models import Word2Vec

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [5]:
import re
import numpy as np
import pandas as pd
from string import punctuation
from PreProcessing.Opentext import get_EBS

['lec00001_EBS.txt', 'lec00002_EBS.txt', 'lec00003_EBS.txt', 'lec00004_EBS.txt', 'lec00005_EBS.txt', 'lec00006_EBS.txt', 'lec00007_EBS.txt', 'lec00008_EBS.txt', 'lec00009_EBS.txt', 'lec00010_EBS.txt', 'lec00011_EBS.txt', 'lec00012_EBS.txt', 'lec00013_EBS.txt', 'lec00014_EBS.txt', 'lec00015_EBS.txt', 'lec00016_EBS.txt', 'lec00017_EBS.txt', 'lec00018_EBS.txt', 'lec00019_EBS.txt', 'lec00020_EBS.txt', 'lec00021_EBS.txt', 'lec00022_EBS.txt', 'lec00023_EBS.txt', 'lec00024_EBS.txt', 'lec00025_EBS.txt', 'lec00026_EBS.txt']


In [6]:
from jamo import h2j, j2hcj, j2h
from soynlp.hangle import levenshtein, jamo_levenshtein
import nltk
from nltk import Text
from nltk.tag import pos_tag
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer, regexp_tokenize
from nltk.help import upenn_tagset
from konlpy.corpus import kolaw, kobill
from konlpy.tag import Kkma, Mecab, Okt, Komoran, Hannanum
from PreProcessing.find_common_part import find_common_part
from PreProcessing.Dictionary import sd, ad, kopro
from PreProcessing.Opentext import get_EBS, get_EBS_entered, get_STT1_Google_entered, get_STT1_Google, get_STT1_Transcribe_entered
from collections import defaultdict
pd.set_option("display.max_rows", None, "display.max_columns", None)
from konlpy.tag import Kkma, Mecab, Okt, Komoran, Hannanum

0
0
0
1
2
5
4
8


In [7]:
EBS = get_EBS(1, 26)

['lec00001_EBS.txt', 'lec00002_EBS.txt', 'lec00003_EBS.txt', 'lec00004_EBS.txt', 'lec00005_EBS.txt', 'lec00006_EBS.txt', 'lec00007_EBS.txt', 'lec00008_EBS.txt', 'lec00009_EBS.txt', 'lec00010_EBS.txt', 'lec00011_EBS.txt', 'lec00012_EBS.txt', 'lec00013_EBS.txt', 'lec00014_EBS.txt', 'lec00015_EBS.txt', 'lec00016_EBS.txt', 'lec00017_EBS.txt', 'lec00018_EBS.txt', 'lec00019_EBS.txt', 'lec00020_EBS.txt', 'lec00021_EBS.txt', 'lec00022_EBS.txt', 'lec00023_EBS.txt', 'lec00024_EBS.txt', 'lec00025_EBS.txt', 'lec00026_EBS.txt']


In [8]:
GOOGLE = get_STT1_Google(1, 26)

['lec00001_STT1_Google.txt', 'lec00002_STT1_Google.txt', 'lec00003_STT1_Google.txt', 'lec00004_STT1_Google.txt', 'lec00005_STT1_Google.txt', 'lec00006_STT1_Google.txt', 'lec00007_STT1_Google.txt', 'lec00008_STT1_Google.txt', 'lec00009_STT1_Google.txt', 'lec00010_STT1_Google.txt', 'lec00011_STT1_Google.txt', 'lec00012_STT1_Google.txt', 'lec00013_STT1_Google.txt', 'lec00014_STT1_Google.txt', 'lec00015_STT1_Google.txt', 'lec00016_STT1_Google.txt', 'lec00017_STT1_Google.txt', 'lec00018_STT1_Google.txt', 'lec00019_STT1_Google.txt', 'lec00020_STT1_Google.txt', 'lec00021_STT1_Google.txt', 'lec00022_STT1_Google.txt', 'lec00023_STT1_Google.txt', 'lec00024_STT1_Google.txt', 'lec00025_STT1_Google.txt', 'lec00026_STT1_Google.txt']


In [9]:
EBS

"반갑습니다, EBS 친구 여러분들. 여러분들과 함께 중학뉴런수학3(하)를 강의하게 된 손석민입니다.2학년 뉴런을 통해서도 만났었는데요. 여러분들, 이렇게 다시 만나게 되어서 정말 반갑습니다.앞으로 3(하) 과정 어떻게 진행되는지에 대해서는 여러분들께서 오리엔테이션 강의를 통해서 확인할 수 있을 거고요.오리엔테이션 강의는 반드시 듣고 오셔야 여러분들이 3학년 과정이, (하) 과정이 특히 어떻게 수업되는지 확인할 수 있을 겁니다.일단 3학년 과정 같은 경우에는 크게 3개의 대단원으로 이루어져 있는데요. 첫 번째가 삼각비라는 단원이고 두 번째가 원의 성질, 세 번째가 통계. 이렇게 이루어져 있습니다.우리가 1학년(하)도 마찬가지고 2학년(하)도 그래 왔듯, ‘주로 도형에 대한 이야기가 3(하)에서 많이 다루어지는구나.’ 라는 걸 생각하면 될 테고요.오늘은 그 첫 시간이니까 ‘삼각비.’ ‘삼각’ 알아요. ‘비’ 알아요. 근데 이제 합쳐져 있는 ‘삼각비’라는 새로운 용어가 나왔죠.그럼 이 개념을 공부하기에 앞서서 당연히 삼각비의 뜻부터 시작하는 것이 첫 시간입니다. 오늘 그 첫 시간, 함께 시작해 보도록 할 텐데요. 중학교 3학년 수학! 손샘과 끝냅시다!오늘 공부할 내용, 바로 두 가지 개념인데요. 오늘 두 가지 개념 첫 번째 제목이 굉장히 길어요. ‘닮은 직각삼각형에서 대응변의 길이의 비’라는 것이 있습니다.그다음 두 번째 개념이 바로 뭐냐면, ‘삼각비’라는 건데 삼각비의 뜻이겠죠? 이 첫 번째 개념은, 앞으로 쭉 공부할 내용이 다 직각삼각형에서 주로 다루어질 텐데직각삼각형에서 갖는 기본적인 성질이 있는데 그 성질을 알고 가자는, ‘삼각비를 공부하기에 앞선 징검다리 내용이다.’ 라고 여러분들이 받아들이시면 됩니다.그렇다면 ‘닮은 직각삼각형에서’라고 했으니까 닮은 직각삼각형을 그려 봐야 되겠죠? 닮은 직각삼각형 두 개를 편하게 그릴 수 있는 제일 좋은 방법은 뭘까 생각해 보면우리는 이렇게 그리면 될 거 같아요. 이렇게 한번 그려 보겠습니다, 이렇게. 여기에서부터 수선

In [10]:
GOOGLE

"반갑습니다 이비에스 여러분들 여러분들과 함께 중학뉴런수학 3할을 강의하기 됨 손성민입니다 2학년 이런을 통해서도 만났었는데 여러분들 이렇게 다시 만나게 되어서 정말 반갑습니다 앞으로 3과정 어떻게 진행되는지에 대해서는 여러분들께서 오리엔테이션 강의를 통해서 확인할 수 있을 거구요 오리엔테이션 강의는 반드시 듣고 오셔야 여러분들이 3학년 과정이 하마 종이접기 어떻게 수업 되는지 확인할 수 있을 겁니다 일단 3학년 가정 같은 경우에는 크게 세 개의대 단원으로 이루어져있는데요 첫 번째가 삼각비라는 단어 이거 두 번째가 원의 성질 세 번째가 통계 이렇게 이루어져 있습니다 우리가 1학년 하도 마찬가지고 2학년 하도 그래왔듯 주로 도형에 대한 이야기가 3화\n많이 다루어 지는구나라는 걸 생각하면 될테구요 오늘은 그 첫 시간이니까 삼각비 삼각 알아요 알아요 근데 이제 합쳐져 있는 삼각비라는 새로운 영화이 개념은 공부하기 앞서서 당연히 삼각비의 뜻부터 시작하는 것이 첫 시간입니다 오늘도 차 시간 함께 시작해 보도록 할 텐데요 중학교 3학년 수학 거 끝냈어\n 오늘 공부한 내용은 바로 두 가지 개념인데요 자 오늘 두 가지 그냥 첫 번째 제목이 굉장히 길어요 닮은 집값 삼각형에서 대응 변의 길이의 피 나는 것이 있습니다 그냥 두 번째 개념이 바로 뭐냐면 나는 건데 생각 뜻이겠죠 2000번째 개념은 두 번째 앞으로 쭉 공부할 내용이다 직각삼각형에서 주로 다루어 질 텐데 직각삼각형에서 가는 기본적인 성질이 있는데 왜 성질을 알고가자 나는 어떤 상각비를 공부하기 앞선 징검다리 내용이라고 여러분들이 받아들이셨습니다 자 그렇다면 닮은 직각삼각형에서 있으니까 닮은 직각삼각형 그려 봐야 되겠죠 닮은 직각 삼각형 두 개를 편하게 그릴 수 있는 제일 좋은 방법은 뭘까 생각해 보면 우리는 이렇게 그러면 될 거 같아요 차 이렇게 한번 그려보겠습니다 이렇게\n 이렇게 여기서부터 수선을 이렇게 커 보겠습니다 일단 바깥쪽에 직각삼각형 않아 보입니다 그죠 또 다른 직각삼각형 하나를 빗변이 1점을 잡아서

In [11]:
def decimalnumber_to_korean(m):
    answer = ''
    for _ in m.group(1):
        answer += kopro[int(_)]
    answer += '점'
    for _ in m.group(3):
        answer += kopro[int(_)]
    return answer

In [12]:
def number_to_korean(s):
    #소수점 한국어로
    s = re.sub(r'(\d+)(\.)(\d+)', decimalnumber_to_korean, s)
    
    #정수 한국어로
    fl = re.findall(r'\d+', s)
    fl = set(fl)
    fl = [int(_) for _ in fl]
    fl.sort(reverse=True)
    fl = [str(_) for _ in fl]
    for n in fl:
        s = s.replace(n, kopro[int(n)])
    return s

In [13]:
def mathsymbol_to_korean(s):
    for ms in sd.keys():
        if ms in s:
            s = s.replace(ms, sd[ms])
    return s

In [14]:
def alphabet_to_korean(s):
    for a in ad.keys():
        if a in s:
            s = s.replace(a, ad[a])
    return s

In [15]:
def remove_newlines(s):
    
    return re.sub(r'[\n]+', ' ', s)

In [16]:
def remove_puncuations(s):
    
    return re.sub(r'[\.?,‘’]+', ' ', s)

In [17]:
def insert_dot(text, dot):
    pattern1 = re.compile(r'(ㅂㄴㅣㄷㅏ)$')   #ㅂ 니다
    pattern2 = re.compile(r'(.ㅔ|ㅏ|ㅓ|ㅐ|ㅗ|ㅜ)(ㅇㅛ)$') # ㅔ요, ㅏ요, ㅓ요
#     pattern3 = re.compile(r'(ㅆ)(ㅈ|ㅊ)(ㅛ)$') 
    pattern4 = re.compile(r'(ㅂ|ㅣ)(ㄴㅣㄲㅏ)$')  #ㅂ니까
    pattern5 = re.compile(r'(ㄴㄷㅔ)$')
    text_list = []
    _1 = 0
    _2 = 0
    _3 = 0
    _4 = 0
    _5 = 0
    _6 = 0
    for _ in text.split(' '):         
        new_ = j2hcj(h2j(_))
        if pattern1.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _1 += 1
        elif pattern2.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _2 += 1
            #list2.append(_)
#         elif pattern3.findall(new_):
#             text_list.append(_.replace(_, _+dot))
#             _3 += 1
#             list3.append(_)
        elif pattern4.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _4 += 1
        elif pattern5.findall(new_):
            text_list.append(_.replace(_,_+dot))
            _5 += 1
        else:
            text_list.append(_)
            _6 += 1
            #list6.append(_)
    print('pattern1 = {}, pattern2= {}, pattern3 = {}, pattern5 = {}, pattern6 = {}'.format(_1,_2,_3,_4,_5,_6)) 
    #print('pattern2 = ',list2)#'\n','list5 = ',list5)
    return text_list    

In [18]:
# EBS 전처리
def EBS_preprocessing(EBS):
    
#     수학기호 한국어로 변환(EBS)
    ebs_math_conversion = mathsymbol_to_korean(EBS)
        
#     숫자 한국어로 변환(EBS)
    ebs_number_conversion = number_to_korean(ebs_math_conversion)
        
#     알파벳 한국어로 변환(EBS)
    ebs_alphabet_conversion = alphabet_to_korean(ebs_number_conversion)
    
#     특수문자 제거
    ebs_punc_removed = remove_puncuations(ebs_alphabet_conversion)
    
#     점 집어넣기
    ebs_dot_inserted = ' '.join(insert_dot(ebs_punc_removed,'.'))

#     띄어쓰기 제거
#     ebs_whitespace_removed = re.sub('[\s]+', '', ebs_dot_inserted)

    return ebs_dot_inserted

In [19]:
# GOOGLE 전처리
def GOOGLE_preprocessing(GOOGLE):
        
#     개행 삭제
    google_newline_removed = remove_newlines(GOOGLE)
    
#     수학기호 한국어로 변환
    google_math_conversion = mathsymbol_to_korean(google_newline_removed)
    
#     숫자 한국어로 변환(GOOGLE)
    google_number_conversion = number_to_korean(google_math_conversion)
        
#     알파벳 한국어로 변환(GOOGLE)
    google_alphabet_conversion = alphabet_to_korean(google_number_conversion)
    
#     특수문자 제거
    google_punc_removed = remove_puncuations(google_alphabet_conversion)
    
#     점 집어넣기
    google_dot_inserted = ' '.join(insert_dot(google_punc_removed,'.'))
    
#     띄어쓰기 제거
#     google_whitespace_removed = re.sub('[\s]+', '', google_dot_inserted)

    return google_dot_inserted

In [20]:
ko_pre_EBS = EBS_preprocessing(EBS)

pattern1 = 1872, pattern2= 2455, pattern3 = 0, pattern5 = 781, pattern6 = 1128


In [21]:
ko_pre_GOOGLE = GOOGLE_preprocessing(GOOGLE)

pattern1 = 1729, pattern2= 2167, pattern3 = 0, pattern5 = 563, pattern6 = 1109


In [22]:
ko_pre_EBS

'반갑습니다.  이비에스 친구 여러분들  여러분들과 함께 중학뉴런수학삼하를 강의하게 된 손석민입니다. 이학년 뉴런을 통해서도 만났었는데요.  여러분들  이렇게 다시 만나게 되어서 정말 반갑습니다. 앞으로 삼하 과정 어떻게 진행되는지에 대해서는 여러분들께서 오리엔테이션 강의를 통해서 확인할 수 있을 거고요. 오리엔테이션 강의는 반드시 듣고 오셔야 여러분들이 삼학년 과정이  하 과정이 특히 어떻게 수업되는지 확인할 수 있을 겁니다. 일단 삼학년 과정 같은 경우에는 크게 삼개의 대단원으로 이루어져 있는데요.  첫 번째가 삼각비라는 단원이고 두 번째가 원의 성질  세 번째가 통계  이렇게 이루어져 있습니다. 우리가 일학년하도 마찬가지고 이학년하도 그래 왔듯   주로 도형에 대한 이야기가 삼하에서 많이 다루어지는구나  라는 걸 생각하면 될 테고요. 오늘은 그 첫 시간이니까.  삼각비   삼각  알아요.   비  알아요.  근데. 이제 합쳐져 있는  삼각비 라는 새로운 용어가 나왔죠 그럼 이 개념을 공부하기에 앞서서 당연히 삼각비의 뜻부터 시작하는 것이 첫 시간입니다.  오늘 그 첫 시간  함께 시작해 보도록 할 텐데요.  중학교 삼학년 수학 손샘과 끝냅시다오늘 공부할 내용  바로 두 가지 개념인데요.  오늘 두 가지 개념 첫 번째 제목이 굉장히 길어요.   닮은 직각삼각형에서 대응변의 길이의 비 라는 것이 있습니다. 그다음 두 번째 개념이 바로 뭐냐면   삼각비 라는 건데. 삼각비의 뜻이겠죠  이 첫 번째 개념은  앞으로 쭉 공부할 내용이 다 직각삼각형에서 주로 다루어질 텐데직각삼각형에서 갖는 기본적인 성질이 있는데. 그 성질을 알고 가자는   삼각비를 공부하기에 앞선 징검다리 내용이다  라고 여러분들이 받아들이시면 됩니다. 그렇다면  닮은 직각삼각형에서 라고 했으니까 닮은 직각삼각형을 그려 봐야 되겠죠  닮은 직각삼각형 두 개를 편하게 그릴 수 있는 제일 좋은 방법은 뭘까 생각해 보면우리는 이렇게 그리면 될 거 같아요.  이렇게 한번 그려 보겠습니다.  이렇게  여기

In [23]:
ko_pre_GOOGLE

'반갑습니다. 이비에스 여러분들 여러분들과 함께 중학뉴런수학 삼할을 강의하기 됨 손성민입니다. 이학년 이런을 통해서도 만났었는데. 여러분들 이렇게 다시 만나게 되어서 정말 반갑습니다. 앞으로 삼과정 어떻게 진행되는지에 대해서는 여러분들께서 오리엔테이션 강의를 통해서 확인할 수 있을 거구요. 오리엔테이션 강의는 반드시 듣고 오셔야 여러분들이 삼학년 과정이 하마 종이접기 어떻게 수업 되는지 확인할 수 있을 겁니다. 일단 삼학년 가정 같은 경우에는 크게 세 개의대 단원으로 이루어져있는데요. 첫 번째가 삼각비라는 단어 이거 두 번째가 원의 성질 세 번째가 통계 이렇게 이루어져 있습니다. 우리가 일학년 하도 마찬가지고 이학년 하도 그래왔듯 주로 도형에 대한 이야기가 삼화 많이 다루어 지는구나라는 걸 생각하면 될테구요. 오늘은 그 첫 시간이니까. 삼각비 삼각 알아요. 알아요. 근데. 이제 합쳐져 있는 삼각비라는 새로운 영화이 개념은 공부하기 앞서서 당연히 삼각비의 뜻부터 시작하는 것이 첫 시간입니다. 오늘도 차 시간 함께 시작해 보도록 할 텐데요. 중학교 삼학년 수학 거 끝냈어  오늘 공부한 내용은 바로 두 가지 개념인데요. 자 오늘 두 가지 그냥 첫 번째 제목이 굉장히 길어요. 닮은 집값 삼각형에서 대응 변의 길이의 피 나는 것이 있습니다. 그냥 두 번째 개념이 바로 뭐냐면 나는 건데. 생각 뜻이겠죠 이천번째 개념은 두 번째 앞으로 쭉 공부할 내용이다 직각삼각형에서 주로 다루어 질 텐데. 직각삼각형에서 가는 기본적인 성질이 있는데. 왜 성질을 알고가자 나는 어떤 상각비를 공부하기 앞선 징검다리 내용이라고 여러분들이 받아들이셨습니다. 자 그렇다면 닮은 직각삼각형에서 있으니까 닮은 직각삼각형 그려 봐야 되겠죠 닮은 직각 삼각형 두 개를 편하게 그릴 수 있는 제일 좋은 방법은 뭘까 생각해 보면 우리는 이렇게 그러면 될 거 같아요. 차 이렇게 한번 그려보겠습니다. 이렇게  이렇게 여기서부터 수선을 이렇게 커 보겠습니다. 일단 바깥쪽에 직각삼각형 않아 보입니다. 그죠 또 다른 

In [24]:
kkma = Kkma()

In [25]:
ebs_pos = kkma.pos(ko_pre_EBS)

In [26]:
google_pos = kkma.pos(ko_pre_GOOGLE)

In [27]:
ebs_pos

[('반갑', 'VA'),
 ('습니다', 'EFN'),
 ('.', 'SF'),
 ('이비', 'NNG'),
 ('에스', 'NNG'),
 ('친구', 'NNG'),
 ('여러분', 'NP'),
 ('들', 'XSN'),
 ('여러분', 'NP'),
 ('들', 'XSN'),
 ('과', 'JKO'),
 ('함께', 'MAG'),
 ('중학', 'NNG'),
 ('뉴런', 'NNG'),
 ('수학', 'NNG'),
 ('삼하', 'NNG'),
 ('를', 'JKO'),
 ('강의', 'NNG'),
 ('하', 'XSV'),
 ('게', 'ECD'),
 ('되', 'VV'),
 ('ㄴ', 'ETD'),
 ('손', 'NNG'),
 ('석민', 'NNG'),
 ('이', 'VCP'),
 ('ㅂ니다', 'EFN'),
 ('.', 'SF'),
 ('이', 'NR'),
 ('학년', 'NNG'),
 ('뉴런', 'NNG'),
 ('을', 'JKO'),
 ('통하', 'VV'),
 ('어서', 'ECD'),
 ('도', 'JX'),
 ('만나', 'VV'),
 ('었', 'EPT'),
 ('었', 'EPT'),
 ('는데요', 'ECD'),
 ('.', 'SF'),
 ('여러분', 'NP'),
 ('들', 'XSN'),
 ('이렇', 'VA'),
 ('게', 'ECD'),
 ('다시', 'MAG'),
 ('만나', 'VV'),
 ('게', 'ECD'),
 ('되', 'VV'),
 ('어서', 'ECD'),
 ('정말', 'MAG'),
 ('반갑', 'VA'),
 ('습니다', 'EFN'),
 ('.', 'SF'),
 ('앞', 'NNG'),
 ('으로', 'JKM'),
 ('삼하', 'NNG'),
 ('과정', 'NNG'),
 ('어떻', 'VA'),
 ('게', 'ECD'),
 ('진행', 'NNG'),
 ('되', 'XSV'),
 ('는', 'ETD'),
 ('지', 'NNB'),
 ('에', 'JKM'),
 ('대하', 'VV'),
 ('어서', 'ECD'),
 

In [29]:
google_pos

[('반갑', 'VA'),
 ('습니다', 'EFN'),
 ('.', 'SF'),
 ('이비', 'NNG'),
 ('에스', 'NNG'),
 ('여러분', 'NP'),
 ('들', 'XSN'),
 ('여러분', 'NP'),
 ('들', 'XSN'),
 ('과', 'JKO'),
 ('함께', 'MAG'),
 ('중학', 'NNG'),
 ('뉴런', 'NNG'),
 ('수학', 'NNG'),
 ('삼', 'NR'),
 ('할', 'NNM'),
 ('을', 'JKO'),
 ('강의', 'NNG'),
 ('하', 'XSV'),
 ('기', 'ETN'),
 ('되', 'VV'),
 ('ㅁ', 'ETN'),
 ('손', 'NNG'),
 ('성', 'XSN'),
 ('민', 'NNG'),
 ('이', 'VCP'),
 ('ㅂ니다', 'EFN'),
 ('.', 'SF'),
 ('이', 'NR'),
 ('학년', 'NNG'),
 ('이렇', 'VV'),
 ('ㄴ', 'ETD'),
 ('읃', 'VV'),
 ('통해서도', 'UN'),
 ('만나', 'VV'),
 ('었', 'EPT'),
 ('었', 'EPT'),
 ('는데', 'ECD'),
 ('.', 'SF'),
 ('여러분', 'NP'),
 ('들', 'XSN'),
 ('이렇', 'VA'),
 ('게', 'ECD'),
 ('다시', 'MAG'),
 ('만나', 'VV'),
 ('게', 'ECD'),
 ('되', 'VV'),
 ('어서', 'ECD'),
 ('정말', 'MAG'),
 ('반갑', 'VA'),
 ('습니다', 'EFN'),
 ('.', 'SF'),
 ('앞', 'NNG'),
 ('으로', 'JKM'),
 ('삼', 'NNG'),
 ('과정', 'NNG'),
 ('어떻', 'VA'),
 ('게', 'ECD'),
 ('진행', 'NNG'),
 ('되', 'XSV'),
 ('는', 'ETD'),
 ('지', 'NNB'),
 ('에', 'JKM'),
 ('대하', 'VV'),
 ('어서', 'ECD'),
 ('는', 

# 준희 여기까지 돌려줘

In [ ]:
google_pos

In [ ]:
def selected_pos(pos_text):
    alist = []
    for _ in pos_text:
        if _[1] in ['NNG','NNP','NNB','NR','NP', 'VV','VA','XR']:
            alist.append(_[0])
            continue
    return alist

# 'NNG' : 보통명사,
# 'NNP': 고유명사,
# 'NNB': 일반의존명사,
# 'NR': 수사,
# 'NP': 대명사, 
# 'VV': 동사,
# 'VA': 형용사,
# 'XR': 어근

In [ ]:
list_ebs_pos = selected_pos(ebs_pos)

In [ ]:
list_google_pos = selected_pos(google_pos)

In [ ]:
list_ebs_pos

In [ ]:
list_google_pos

In [ ]:
integrated_into_W2V = [list_ebs_pos + list_google_pos]

In [ ]:
type(integrated_into_W2V)

In [ ]:
model_skip = Word2Vec(
    integrated_into_W2V, 
    size = 100,
    sg = 1
)
words_skip = list(model_skip.wv.vocab)
print(words_skip)

In [ ]:
model_skip.wv.similar_by_word('육이구',topn=10)

In [ ]:
EBSlist_tokens = [' '.join(Text(ma(_)).tokens) for _ in new_textE1]

In [ ]:
EBSlist_tokens

In [ ]:
t = Tokenizer()
t.fit_on_texts(morphed_EBS)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

In [ ]:
sequences = list()

for line in morphed_EBS: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

In [ ]:
print(t.word_index)

In [ ]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

In [ ]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

In [ ]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

In [ ]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [ ]:
print(X[:3])

In [ ]:
print(y[:3]) # 레이블

In [ ]:
vocab_size

In [ ]:
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=10, verbose=2)

In [ ]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [ ]:
sentence_generation(model, t, '이 병 분에 높이', 5)